Imported files

In [1]:
import inline as inline
import matplotlib
import requests, six
import lxml.html as lh
import pandas as pd
from matplotlib import colors
from itertools import cycle, islice
import matplotlib.pyplot as plt
#% matplotlib inline

Gather URL and associated information

In [2]:
url = "https://www.pro-football-reference.com/years/2020/fantasy.htm"
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

Iterate through the table elements from url, store the heading and a list of data in tuples

In [6]:
col = []
i = 0
heading = []
for headings in tr_elements[1]:
    i+=1
    name = headings.text_content()
    col.append((name, []))
    heading.append(name)
heading[10]='PassTD'
heading[15]='RushTD'
heading[20]='RecTD'
print(heading[15])

RushTD


Add the data to the beforementioned list. Has some data cleansing for blank inputs and the repeated headings in this case

In [7]:
for j in range(1, len(tr_elements)):
    cells = tr_elements[j]
    i = 0

    for t in cells.iterchildren():
        data = t.text_content()
        if data == '':
            data = 0
        if i > 0:
            try:
                data = float(data)
            except:
                pass
        #every 30 or so lines, the headings repeated. This checks to make sure those headings don't get into the data
        if data not in heading:
            col[i][1].append(data)
            i += 1

create a dictionary to store the data and display it using a pandas dataframe

In [8]:
Dict={title:column for (title,column) in col}
pd.set_option('display.max_columns', None)
df=pd.DataFrame(Dict)

df.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,TD,Int,Y/A,Tgt,Rec,Y/R,Fmb,FL,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
0,1,Josh Jacobs,LVR,RB,22.0,1.0,1.0,0.0,25.0,46.0,3.0,0.0,3.72,6.0,4.0,11.50,0.0,0.0,0.0,0.0,32.0,35.9,35.9,33.9,22.0,1.0,1.0
1,2,Davante Adams,GNB,WR,28.0,1.0,1.0,0.0,0.0,156.0,2.0,0.0,0.00,17.0,14.0,11.14,0.0,0.0,0.0,0.0,28.0,41.6,44.6,34.6,20.0,1.0,2.0
2,3,Adam Thielen,MIN,WR,30.0,1.0,1.0,0.0,0.0,110.0,2.0,0.0,0.00,8.0,6.0,18.33,0.0,0.0,1.0,0.0,25.0,31.0,34.0,28.0,17.0,2.0,3.0
3,4,Calvin Ridley,ATL,WR,26.0,1.0,1.0,0.0,1.0,130.0,2.0,0.0,-1.00,12.0,9.0,14.44,0.0,0.0,0.0,0.0,25.0,33.9,36.9,29.4,17.0,3.0,4.0
4,5,Christian McCaffrey,CAR,RB,24.0,1.0,1.0,0.0,23.0,38.0,2.0,0.0,4.17,4.0,3.0,12.67,0.0,0.0,0.0,0.0,25.0,28.4,28.4,26.9,16.0,2.0,5.0


Backup with json file for future reference

In [9]:
df.to_json('FantasyFootballWeekOneData.json')

In [10]:
def max_stat(df, all_col):
    tlist = []
    for i in range(4, 33):
        tlist.append(all_col[i])
    for col in tlist:
        output=''
        stat = df[col].max()
        name = df[df[col]==df[col].max()]['Player'].values[0]
        output += name+' has the best '+ col + ' of '+str(stat)+'.\n'
        print(output)
        
def min_stat(df, all_col):
    tlist = []
    for i in range(4, 33):
        tlist.append(all_col[i])
    for col in tlist:
        output=''
        stat = df[col].min()
        name = df[df[col]==df[col].min()]['Player'].values[0]
        output += name+' has the lowest '+ col + ' of '+str(stat)+'.\n'
        print(output)

In [11]:
max_stat(df, heading)
print()
min_stat(df, heading)

Tom Brady  has the best Age of 43.0.

Josh Jacobs has the best G of 1.0.

Josh Jacobs has the best GS of 1.0.

Matt Ryan has the best Cmp of 37.0.

Derrick Henry  has the best Att of 31.0.

Julio Jones  has the best Yds of 157.0.



KeyError: 'PassTD'